# Installing Necessary Packages

In [ ]:
pip install requests
pip install geocoder

# Importing Necessary Libraries

In [78]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#conda config --add channels conda-forge
#conda install -c conda-forge geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
from googlemaps import Client
from geopy.geocoders import GoogleV3

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#conda install -c conda-forge folium=0.5.0 # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd
from pandas.io.json import json_normalize
print('Libraries imported.')


Libraries imported.


In [ ]:
g = geocoder.google('Karachi, Pakistan')
print(g.latlng)
                    

# Reading Dataframe into Pandas

In [2]:
df = pd.read_csv('isl_areas.csv', names =['Neighbourhood'], header=None)

In [3]:
df.shape

#We have 301 neighborhoods in the list

(301, 1)

In [4]:
df.head()

,Neighbourhood
0,Abbas Town
1,Abbasi Shaheed
2,Abdul Rehman Goth
3,Abdullah Goth
4,Abidabad


# Getting coordinates for Karachi

In [96]:
city = 'Karachi, Pakistan'
address = city
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode(city)
khi_lat = location.latitude
khi_lng = location.longitude
print(address, khi_lat,':',khi_lng)

Karachi, Pakistan 25.1446897 : 67.1847767315734


# Getting coordinates for the Neighborhoods in the dataframe

In [5]:
latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array

for nbd in df["Neighbourhood"]: 
    try:
        place_name = nbd + " , Karachi" 
        geolocator = Nominatim(user_agent="specify_your_app_name_here")
        location = geolocator.geocode(place_name)
        lat = location.latitude
        lng = location.longitude
    
        latitudes.append(lat) # Appending to the list of latitudes
        longitudes.append(lng) # Appending to the list of longitudes
        
    except Exception as e:
        latitudes.append("")
        longitudes.append("")

In [6]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [11]:
df3 = df[df['Latitude']==""] # creating a dataframe of empty coordinates from the original dataframe

In [99]:
df3.count()

Neighbourhood    89
Latitude         89
Longitude        89
dtype: int64

##### There are 89 empty geocordinates

# Removing empty geocordinate rows

In [19]:
df=df[df.Latitude != ""]

In [21]:
df.shape

#Our final dataframe has 212 neighbourhoods

(212, 3)

# Creating map of Karachi and marking neighbourhoods

In [22]:
# Lahore latitude and longitude using Google search
khi_lat = 25.1446897
khi_lng = 67.1847767315734
 
# Creates map of Lahore using latitude and longitude values
map_isl = folium.Map(location=[khi_lat, khi_lng], zoom_start=10)

# Add markers to map
for lat, lng, area in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_isl)  
    
map_isl

# Mapping venues against neighborhoods using FourSquare API

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
CLIENT_ID = 'OPGHRGPHQSVZ0MDLQAXSPPFQO2X5SPIUMHTKMCPX3150QDW3' # your Foursquare ID
CLIENT_SECRET = 'CR1FOU0MWNCHIKD1UMVB3K0HJSFDWVLI1XX3PRGNSTTKVT2O' # your Foursquare Secret
VERSION = '20191130' # Foursquare API version

def getNearbyVenues(names, latitudes, longitudes, radius=2000, LIMIT=10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
khi_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Abbas Town
Abbasi Shaheed
Abdul Rehman Goth
Abdullah Goth
Abu Zar Ghaffari
Afridi Colony
Ahsanabad
Akhtar Colony
Allama Iqbal Colony
Ancholi
Arbian
Awami Colony
Azam Basti
Bahadurabad
Baldia Colony
Baloch Colony
Baloch Goth
Banaras Colony
Baradari, Pakistan
Bath Island
Bhittai Colony
Bihar Colony
Bilal Colony
Bismillah Chowk
Buffer Zone
Catholic Colony
Cattle Colony
Central Jacob Lines
Chakiwara
Chanesar Goth
Chittagong Colony
City Railway Colony
Civil Line
Clifton
Dak Khana
Darakhshan
Darvesh Goth
Dastagir
Dastagir Colony
Data Nagar
Dawood Chowrangi
Defence Housing Authority
Defence View
Delhi Colony
DHA City
Dhoraji Colony
Drigh Colony
F.B.Area (Gulberg Town)
Faisal Colony
Farooq-e-Azam
Firdous Colony
Frontier Colony
Gabol Colony
Gadap
Garden East
Garden West
Garden
Gazdarabad
Gharibabad
Ghausia Colony
Ghaziabad
Gizri
Godhra
Gole Market Nazimabad
Golimar
Goth Lashkari
Goth Mohammad Ali
Goth Shaikhan
Green Park City
Gujrat Colony
Gulbahar
Gulistan-e-Johar
Gulistan-e-Zafar
Gulshan-e-Am

In [33]:
print(khi_venues.shape)
khi_venues.head()

#We have 1326 venues retrieved from FourSquare againt our neighourhood list

(1326, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbas Town,25.002187,67.069156,Air Asia International Travel & Tours,25.000294,67.065654,Transportation Service
1,Abbas Town,25.002187,67.069156,Warsi Autos,25.003157,67.079828,Motorcycle Shop
2,Abbas Town,25.002187,67.069156,Fruit Market,24.999861,67.080788,Farmers Market
3,Abbas Town,25.002187,67.069156,Pizza Point,24.988988,67.060268,Pizza Place
4,Abbasi Shaheed,24.920251,67.028837,Dunkin',24.926934,67.033014,Donut Shop


In [35]:
khi_venues.groupby('Neighbourhood').count()

#Grouping the neighbourhoods

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Abbas Town,4,4,4,4,4,4
Abbasi Shaheed,10,10,10,10,10,10
Abdul Rehman Goth,2,2,2,2,2,2
Abu Zar Ghaffari,10,10,10,10,10,10
Afridi Colony,1,1,1,1,1,1
Ahsanabad,3,3,3,3,3,3
Akhtar Colony,10,10,10,10,10,10
Allama Iqbal Colony,10,10,10,10,10,10
Ancholi,10,10,10,10,10,10


In [37]:
print('There are {} uniques categories.'.format(len(khi_venues['Venue Category'].unique())))

There are 117 uniques categories.


# Printing a list of unique categories

In [38]:
khi_venues['Venue Category'].unique()[:50]

array(['Transportation Service', 'Motorcycle Shop', 'Farmers Market',
       'Pizza Place', 'Donut Shop', 'Diner', 'Juice Bar',
       'Fast Food Restaurant', 'Shopping Mall', 'Food Court',
       'Department Store', 'Bakery', 'Sandwich Place', 'Dessert Shop',
       'Beach', 'African Restaurant', 'Steakhouse', 'Burger Joint',
       'Mobile Phone Shop', 'Restaurant', 'BBQ Joint', 'History Museum',
       'Mediterranean Restaurant', 'Asian Restaurant',
       'Mexican Restaurant', 'Coffee Shop', 'Portuguese Restaurant',
       'Clothing Store', 'Market', 'Park', 'Indian Restaurant',
       'Movie Theater', 'Cricket Ground', 'Pakistani Restaurant',
       'Resort', 'Italian Restaurant', 'Salad Place',
       'Furniture / Home Store', 'Tea Room', 'Snack Place', 'Platform',
       'Bus Station', 'Gym', 'Café', 'Frozen Yogurt Shop', 'Social Club',
       'Japanese Restaurant', 'Chinese Restaurant', 'Golf Course',
       'Event Service'], dtype=object)

In [39]:
# one hot encoding
khi_onehot = pd.get_dummies(khi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
khi_onehot['Neighbourhood'] = khi_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [khi_onehot.columns[-1]] + list(khi_onehot.columns[:-1])
khi_onehot = khi_onehot[fixed_columns]

khi_onehot.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Beach,Boat or Ferry,Bowling Alley,Breakfast Spot,Bridal Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Cricket Ground,Cruise,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,English Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hockey Field,Hookah Bar,Hospital,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Music Store,Night Market,Noodle House,Outdoors & Recreation,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Portuguese Restaurant,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Sporting Goods Shop,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Tea Room,Thai Restaurant,Theater,Train Station,Transportation Service,Women's Store
0,Abbas Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Abbas Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abbas Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abbas Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Abbasi Shaheed,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
khi_onehot.shape

(1326, 118)

In [43]:
khi_grouped = khi_onehot.groupby('Neighbourhood').mean().reset_index()
khi_grouped

,Neighbourhood,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Beach,Boat or Ferry,Bowling Alley,Breakfast Spot,Bridal Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Cricket Ground,Cruise,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,English Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hockey Field,Hookah Bar,Hospital,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Music Store,Night Market,Noodle House,Outdoors & Recreation,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Portuguese Restaurant,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Sporting Goods Shop,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Tea Room,Thai Restaurant,Theater,Train Station,Transportation Service,Women's Store
0,Abbas Town,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.250000,0.00,0.0,0.00,0.0,0.00,0.000000,0.000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.25,0.00
1,Abbasi Shaheed,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.100000,0.000000,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000,0.100000,0.100000,0.10,0.100,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.100000,0.00,0.000000,0.00,0.000000,0.1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.100000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.0,0.00,0.000000,0.000,0.0,0.000,0.100,0.000000,0.100000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.00,0.00
2,Abdul Rehman Goth,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.0,0.00,0.000000,0.000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,

# Selecting Shopping Malls from our venues list

In [44]:
len(khi_grouped[khi_grouped["Shopping Mall"] > 0])

26

##### There are 26 shopping malls in the neighborhood list

In [46]:
khi_mall = khi_grouped[["Neighbourhood","Shopping Mall"]]

In [47]:
khi_mall.head(30)

,Neighbourhood,Shopping Mall
0,Abbas Town,0.0
1,Abbasi Shaheed,0.1
2,Abdul Rehman Goth,0.0
3,Abu Zar Ghaffari,0.1
4,Afridi Colony,0.0
5,Ahsanabad,0.0
6,Akhtar Colony,0.0
7,Allama Iqbal Colony,0.0
8,Ancholi,0.1
9,Arbian,0.0


# Clustering the Malls using K Means

In [50]:
khi_mall_clustering = khi_mall.drop('Neighbourhood', 1)

In [60]:
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(khi_mall_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0])

In [61]:
khi_merged = khi_mall.copy()

# add clustering labels
khi_merged["Cluster Labels"] = kmeans.labels_

In [62]:
khi_merged.head()

,Neighbourhood,Shopping Mall,Cluster Labels
0,Abbas Town,0.0,0
1,Abbasi Shaheed,0.1,1
2,Abdul Rehman Goth,0.0,0
3,Abu Zar Ghaffari,0.1,1
4,Afridi Colony,0.0,0


In [63]:
khi_merged = khi_merged.join(df.set_index("Neighbourhood"), on="Neighbourhood")

print(khi_merged.shape)
khi_merged.head() 

(192, 5)


,Neighbourhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Abbas Town,0.0,0,25.0022,67.0692
1,Abbasi Shaheed,0.1,1,24.9203,67.0288
2,Abdul Rehman Goth,0.0,0,24.8513,66.8253
3,Abu Zar Ghaffari,0.1,1,24.9416,67.0324
4,Afridi Colony,0.0,0,24.9213,66.9758


In [64]:
# sort the results by Cluster Labels
print(khi_merged.shape)
khi_merged.sort_values(["Cluster Labels"], inplace=True)
khi_merged

(192, 5)


,Neighbourhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Abbas Town,0.000000,0,25.0022,67.0692
115,Moosa Lane,0.000000,0,24.8571,66.9973
116,Mubarak Goth,0.000000,0,24.854,67.0002
117,Mujahid Colony,0.000000,0,24.9005,67.1036
118,Mujahidabad,0.000000,0,24.9376,67.007
119,Muslim Mujahid Colony,0.000000,0,24.8908,67.1416
121,Muslimabad,0.000000,0,24.8777,67.0498
122,Mustufa Colony,0.000000,0,24.8858,67.1441
123,Muzaffarabad,0.000000,0,24.8292,67.2299
114,Mominabad,0.000000,0,24.9305,66.9917


# Creating a map again with clusters of malls

In [65]:
# create map
map_clusters = folium.Map(location=[khi_lat, khi_lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(khi_merged['Latitude'], khi_merged['Longitude'], khi_merged['Neighbourhood'], khi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Cluster 0 is represented by Red circles, Cluster 1 with purple and Cluster 2 with mint

# Analysing Different Clusters

In [73]:
khi_merged.loc[khi_merged['Cluster Labels'] == 0, khi_merged.columns[[0] + list(range(2, khi_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,Latitude,Longitude
0,Abbas Town,0,25.0022,67.0692
115,Moosa Lane,0,24.8571,66.9973
116,Mubarak Goth,0,24.854,67.0002
117,Mujahid Colony,0,24.9005,67.1036
118,Mujahidabad,0,24.9376,67.007
119,Muslim Mujahid Colony,0,24.8908,67.1416
121,Muslimabad,0,24.8777,67.0498
122,Mustufa Colony,0,24.8858,67.1441
123,Muzaffarabad,0,24.8292,67.2299
114,Mominabad,0,24.9305,66.9917


In [74]:
khi_merged.loc[khi_merged['Cluster Labels'] == 1, khi_merged.columns[[0] + list(range(2, khi_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,Latitude,Longitude
189,Yaseenabad,1,24.9199,67.0762
1,Abbasi Shaheed,1,24.9203,67.0288
8,Ancholi,1,24.9457,67.0767
3,Abu Zar Ghaffari,1,24.9416,67.0324
185,Turk Colony,1,24.9291,67.1036
86,Junaid Nagar,1,24.9208,67.0951
75,Hasrat Mohani Colony,1,24.936,67.0301
134,Nursery,1,24.9358,67.0311
36,Dastagir Colony,1,24.9232,67.0745
132,North Karachi,1,24.9725,67.067


In [75]:
khi_merged.loc[khi_merged['Cluster Labels'] == 2, khi_merged.columns[[0] + list(range(2, khi_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,Latitude,Longitude
120,Muslim Town,2,24.9683,67.0722
52,Garden West,2,24.8803,67.0203
71,Gulzar-e-Hijri,2,24.9488,67.0887
